In [0]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


# create data VGG
## import libraries

In [0]:
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Flatten, ActivityRegularization, Lambda
from keras.layers import Convolution2D, MaxPooling2D, Conv1D, MaxPooling1D, TimeDistributed
from keras.layers import AveragePooling2D, Input
from keras.utils import np_utils, normalize
from keras.engine import InputLayer
from keras import backend as K

import os
import h5py
import time

import sklearn
from sklearn.preprocessing import normalize
import random

from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix


import matplotlib.pyplot as plt
import numpy as np
import time
import math

Using TensorFlow backend.


## modifiable variables

In [0]:
# dataset's name 
doc = 'chest_xray'
# if the dataset is splited
is_splited = True
# test size for dataset unsplited
test_size = 0.2
# if a dataset contains validation set
has_val = False

# paths to load datasets and to save the model
dir_path = 'drive/My Drive/medical_image_recognition/datasets/'
save_path = 'drive/My Drive/medical_image_recognition/notebooks_Chen/'
model_path = 'drive/My Drive/medical_image_recognition/models/'

# paths
# vgg_fold='datanp_vgg/'
vgg_fold='vgg19_trained/'
kmeans_fold="kmeans_PCA/"
vlad_fold="vlad_fold/"

# model
flag_load_model = True
model_with_structure = True
# model type name and weights' path
model_name = 'vgg19' # vgg19, vgg19custom, vgg16, vgg16custom
model_file_name = 'model_1_model.hdf5'

# feature extraction parameters
scales=[2]
layers=['fc2']
pooling = 'max'

# if save vectors in differents files in case the RAM is insufficient
vgg_multi_files = True
RAM_SIZE = 8 * 1024 * 1024 * 1024 #GPU RAM in Byte

flag_VLAD = False

#Kmeans
k_descriptors=30

print('Done')

Done


## file path check

In [0]:
dir_path_DataSet = dir_path + doc + '/'
save_path_Dataset = save_path + doc + '/'

vgg_path = save_path_Dataset+'/'+vgg_fold
kmeans_path = save_path_Dataset+'/'+ "kmeans_PCA/"
vlad_path = save_path_Dataset+'/'+ "vlad_fold/"
model_file_path = model_path + doc + '/' + model_file_name

os.system('mkdir -p {}'.format(vgg_path))
if flag_VLAD:
  os.system('mkdir -p {} {}'.format(kmeans_path, vlad_path))
  
if flag_load_model:
  if not os.path.exists(model_file_path):
    print('model file not existe')
  else:
    print('Done')

model file not existe


## image process functions

In [0]:
def read_dir(dir_path):
  """return the differents categories, the images in these categories, and the number of images per category"""
  listDir = sorted(os.listdir(dir_path))
  size = 0
  listFiles = []
  Y = []
  for d in listDir :
    files = sorted(os.listdir(dir_path+'/'+d))
    size += len(files)
    files = [dir_path+'/'+d+'/'+f for f in files]
    Y += [d] * len(files)
    listFiles += files
  return listFiles, Y, size

def dir_train_test_split(listFiles, Y):
  random.seed(42)
  test_index = random.sample(list(range(len(listFiles))), int(test_size*len(listFiles)))
  train_index = [x for x in list(range(len(listFiles))) if x not in test_index]
  
  listFiles_train = [listFiles[i] for i in train_index]
  listFiles_test = [listFiles[i] for i in test_index]
  Y_train = [Y[i] for i in train_index]
  Y_test = [Y[i] for i in test_index]
  return listFiles_train, Y_train, listFiles_test, Y_test

def create_zero_array(layer, n, nb_images_left, nbImagesPerFile):
  """return a numpy array with zeros and with the needed scales"""
  nb = min(nb_images_left, nbImagesPerFile)
  if layer == 'block5_pool' or layer == 'vgg19':
    X = np.zeros((nb, 7 + 10*n, 7 + 10*n, 512))
  else :
    if n == 0 :
      X = np.zeros((nb, 4096))
#       X = np.zeros((nb, 512))
    else :
#       X = np.zeros((nb, 1 + 10*n, 1 + 10*n, 4096))
      X = np.zeros((nb, 1 + 20*n, 1 + 20*n, 512))
  return X


def complete_array_cancer(files, X_new, Y, n, n_image) :
  """for all the images in listFiles, extract the features thanks to the network"""
  """return a 4-dimensional array with a 3-dimensional array per image"""
  nb = 0
  cpt = 0
  t0 = time.time()
  print(len(files))
  nb += len(files)
  Y_new = []
  cpt_X = 0
  for f in files :
    if cpt < n_image:
      cpt += 1
      continue
    if cpt_X == X_new.shape[0]:
        break
    t1 = time.time()
    img = image.load_img(f, target_size=(224 + 320*n, 224 + 320*n))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    X_new[cpt_X,:] = features[0,:]
    Y_new.append(Y[cpt])
    cpt += 1
    cpt_X += 1
    t2 = time.time()
    print('complete array{}/{}, time: {}s'.format(cpt,nb, t2-t1))
  print('total', t2 - t0)
  return X_new, Y_new

def to_fully_conv(model):
  """transforms the Convolutional Neural Network model into a Fully Convolutional Network"""

  new_model = Sequential()
  input_layer = InputLayer(input_shape=(None, None, 3), name="input_new")
  new_model.add(input_layer)

  for layer in model.layers:

    if "Flatten" in str(layer):
      flattened_ipt = True
      f_dim = layer.input_shape
      continue

    elif "Dense" in str(layer):

      input_shape = layer.input_shape
      output_dim =  layer.get_weights()[1].shape[0]
      W,b = layer.get_weights()

      if flattened_ipt:
          shape = (f_dim[1],f_dim[2],f_dim[3],output_dim)
          new_W = W.reshape(shape)
          aa = np.array([new_W,b])
          new_layer = Convolution2D(output_dim,(f_dim[1],f_dim[2]),
                                    activation=layer.activation,
                                    padding='valid', weights=[new_W,b])
          flattened_ipt = False

      else:
          shape = (1,1,input_shape[1],output_dim)
          new_W = W.reshape(shape)
          new_layer = Convolution2D(output_dim,(1,1),
                                    activation=layer.activation,
                                    padding='valid', weights=[new_W,b])
    else:
      new_layer = layer
    new_model.add(new_layer)
    
  return new_model

def pool(X, p):
  """return the numpy array X on which the pooling p was performed"""
  if(len(X.shape) == 2):
    return X
  if p == 'max' :  
    X_pool = np.max(X, axis = (1, 2)) 
  elif p == 'mean' :
    X_pool = np.mean(X, axis = (1, 2)) 
  else :
    X_pool = X
  return X_pool

def cal_nbSaveFiles(layer, n, size_train, size_test, size_val = 0):
  if layer == 'block5_pool':
    size_image = 8*(7+10*n)*(7+10*n)*512
  else:
    size_image = 8*(1+10*n)*(1+10*n)*4096
    
  nbSaveFiles_train = math.ceil(size_image * size_train / RAM_SIZE * 2)
  nbSaveFiles_test = math.ceil(size_image * size_test / RAM_SIZE * 2)
  nbSaveFiles_val = math.ceil(size_image * size_val / RAM_SIZE * 2)
  nbImagesPerFile = int(RAM_SIZE / 2 // size_image)
  return nbImagesPerFile, nbSaveFiles_train, nbSaveFiles_test, nbSaveFiles_val


#path where the output array will be saved
def get_vector_files_path(flag_VLAD, vgg_multi_files, save_path, doc, layer, n, pooling, nbSaveFiles_train, nbSaveFiles_test, nbSaveFiles_val):
  
  path_save_train = []
  path_save_test = []
  path_save_val = []
  path_save_Y_train = []
  path_save_Y_test = []
  path_save_Y_val = []
    
  if flag_VLAD == False:
    for i in range(nbSaveFiles_train):
      path_save_train.append(  save_path + 'X_train_' + layer + '_' + pooling + '(' + str(n) + ')'+ str(i)+'.npy')
      path_save_Y_train.append(save_path + 'y_train_' + layer + '_' + pooling + '(' + str(n) + ')'+ str(i)+'.npy')
    for i in range(nbSaveFiles_test):
      path_save_test.append(  save_path + 'X_test_' + layer + '_' + pooling + '(' + str(n) + ')'+ str(i)+'.npy')
      path_save_Y_test.append(save_path + 'y_test_' + layer + '_' + pooling + '(' + str(n) + ')'+ str(i)+'.npy')
    for i in range(nbSaveFiles_val):
      path_save_val.append(  save_path + 'X_val_' + layer + '_' + pooling + '(' + str(n) + ')'+ str(i)+'.npy')
      path_save_Y_val.append(save_path + 'y_val_' + layer + '_' + pooling + '(' + str(n) + ')'+ str(i)+'.npy')
  else:
    if vgg_multi_files:
      for i in range(nbSaveFiles_train):
        path_save_train.append(  save_path + 'X_train_' + layer + '(' + str(n) + ')'+ str(i)+'.npy')
        path_save_Y_train.append(save_path + 'y_train_' + layer + '(' + str(n) + ')'+str(i)+'.npy')
      for i in range(nbSaveFiles_test):
        path_save_test.append(  save_path + 'X_test_' + layer + '(' + str(n) + ')' + str(i)+'.npy')
        path_save_Y_test.append(save_path + 'y_test_' + layer + '(' + str(n) + ')' + str(i)+'.npy')
      for i in range(nbSaveFiles_val):
        path_save_val.append(  save_path + 'X_val_' + layer + '(' + str(n) + ')' + str(i)+'.npy')
        path_save_Y_val.append(save_path + 'y_val_' + layer + '(' + str(n) + ')' + str(i)+'.npy')
  return path_save_train, path_save_Y_train, path_save_test, path_save_Y_test, path_save_val, path_save_Y_val

print('Done')

Done


## costum neural network definition

In [0]:
def load_full_model(model_file_path):
  model = load_model(model_file_path)
  return model

def get_cnn_model(model_name: str):
    """
    TODO
    """
    if model_name == 'vgg16':
        return create_vgg16()
    elif model_name == 'vgg16custom':
        return create_vgg16custom()
    elif model_name == 'vgg19':
        return create_vgg19()
    elif model_name == 'vgg19custom':
        return create_vgg19custom()
    else:
        raise ValueError(
                'The model {} isn\'t implemented yet'.format(model_name))

################################################################################
def create_vgg16():
    """
    TODO
    """
    base_model = VGG16(include_top=False,
                             weights='imagenet',
                             input_shape=(HEIGHT, WIDTH, CHANNELS))

    model = Sequential()
    model.add(base_model)

    # add classification block
    model.add(Flatten(name='flatten'))
    model.add(Dense(1024, activation='relu', name='fc1'))
    model.add(Dropout(0.7, name='dropout1'))
    model.add(Dense(512, activation='relu', name='fc2'))
    model.add(Dropout(0.5, name='dropout2'))
    model.add(Dense(NUM_LABELS, activation='softmax', name='predictions'))

    # see the base model architecture
    #base_model.summary()

    return model

################################################################################
def create_vgg16custom():
    """
    TODO
    """
    img_input = Input(shape=(HEIGHT, WIDTH, CHANNELS), name='ImageInput')

    # block 1
    x = Conv2D(64, (3, 3),
               activation='relu',
               padding='same',
               name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3),
               activation='relu',
               padding='same',
               name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), name='block1_pool')(x)

    #TODO
    # variables to change for SeparableConv2D
    # no activation in xception
    # use_bias=False

    # block 2
    x = SeparableConv2D(128, (3, 3),
                        activation='relu',
                        padding='same',
                        name='block2_conv1')(x)
    x = SeparableConv2D(128, (3, 3),
                        activation='relu',
                        padding='same',
                        name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), name='block2_pool')(x)

    # block 3
    x = SeparableConv2D(256, (3, 3),
                        activation='relu',
                        padding='same',
                        #use_bias=False,
                        name='block3_conv1')(x)
    x = BatchNormalization(name='block3_conv1_bn')(x)
    x = SeparableConv2D(256, (3, 3),
                        activation='relu',
                        padding='same',
                        #use_bias=False,
                        name='block3_conv2')(x)
    x = BatchNormalization(name='block3_conv2_bn')(x)
    x = SeparableConv2D(256, (3, 3),
                        activation='relu',
                        padding='same',
                        name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), name='block3_pool')(x)

    # block 4
    x = SeparableConv2D(512, (3, 3),
                        activation='relu',
                        padding='same',
                        name='block4_conv1')(x)
    x = BatchNormalization(name='block4_conv1_bn')(x)
    x = SeparableConv2D(512, (3, 3),
                        activation='relu',
                        padding='same',
                        name='block4_conv2')(x)
    x = BatchNormalization(name='block4_conv2_bn')(x)
    x = SeparableConv2D(512, (3, 3),
                        activation='relu',
                        padding='same',
                        name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), name='block4_pool')(x)

    # block 5
    # TODO

    # classification block
    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu', name='fc1')(x) #4096
    x = Dropout(0.7, name='dropout1')(x) # not present
    x = Dense(512, activation='relu', name='fc2')(x) # 4096
    x = Dropout(0.5, name='dropout2')(x) # not present
    x = Dense(NUM_LABELS, activation='softmax', name='predictions')(x)

    # create the cnn
    model = Model(img_input, x, name='vgg16_custom')

    # !!! WEIGHTS !!!
    weights_path = MODEL_PATH + 'weights_notop.h5'
    print('Trying to load weights from {} ...'.format(weights_path))
    
    ### TODO cleaner
    from pathlib import Path
    weights_path = Path(weights_path)
    if not weights_path.is_file():
        vgg16_model = VGG16(weights='imagenet', include_top=False)
        vgg16_model.save_weights(weights_path)

    # load weights
    model.load_weights(weights_path, by_name=True)
    print('DONE')
    return model

################################################################################
def create_vgg19():
    """
    TODO
    """
    base_model = VGG19(include_top=False,
                             weights='imagenet',
                             input_shape=(HEIGHT, WIDTH, CHANNELS))

    model = Sequential()
    model.add(base_model)

    # add classification block
    model.add(Flatten(name='flatten'))
    model.add(Dense(1024, activation='relu', name='fc1'))
    model.add(Dropout(0.7, name='dropout1'))
    model.add(Dense(512, activation='relu', name='fc2'))
    model.add(Dropout(0.5, name='dropout2'))
    model.add(Dense(NUM_LABELS, activation='softmax', name='predictions'))

    #base_model.summary()
    #model.summary()

    return model

################################################################################
def create_vgg19custom():
    """
    TODO
    """
    
    #TODO
    # variables to change for SeparableConv2D
    # no activation in xception
    # use_bias=False
    
    img_input = Input(shape=(HEIGHT, WIDTH, CHANNELS))

    # block 1
    # it's exactly the same as in vgg19
    x = Conv2D(64, (3, 3),
               activation='relu',
               padding='same',
               name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3),
               activation='relu',
               padding='same',
               name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # block 2
    x = SeparableConv2D(128, (3, 3),
                        activation='relu',
                        padding='same',
                        name='block2_conv1')(x)
    x = SeparableConv2D(128, (3, 3),
                        activation='relu',
                        padding='same',
                        name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), name='block2_pool')(x)

    # block 3
    x = SeparableConv2D(256, (3, 3),
                        activation='relu',
                        padding='same',
                        #use_bias=False,
                        name='block3_conv1')(x)
    x = BatchNormalization(name='block3_conv1_bn')(x)
    x = SeparableConv2D(256, (3, 3),
                        activation='relu',
                        padding='same',
                        #use_bias=False,
                        name='block3_conv2')(x)
    x = BatchNormalization(name='block3_conv2_bn')(x)
    x = SeparableConv2D(256, (3, 3),
                        activation='relu',
                        padding='same',
                        name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), name='block3_pool')(x)

    # block 4
    x = SeparableConv2D(512, (3, 3),
                        activation='relu',
                        padding='same',
                        name='block4_conv1')(x)
    x = BatchNormalization(name='block4_conv1_bn')(x)
    x = SeparableConv2D(512, (3, 3),
                        activation='relu',
                        padding='same',
                        name='block4_conv2')(x)
    x = BatchNormalization(name='block4_conv2_bn')(x)
    x = SeparableConv2D(512, (3, 3),
                        activation='relu',
                        padding='same',
                        name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), name='block4_pool')(x)

    # block 5
    # TODO

    # classification block
    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu', name='fc1')(x) #4096
    x = Dropout(0.7, name='dropout1')(x) # not present
    x = Dense(512, activation='relu', name='fc2')(x) # 4096
    x = Dropout(0.5, name='dropout2')(x) # not present
    x = Dense(NUM_LABELS, activation='softmax', name='predictions')(x)

    #TODO load weights here

    model = Model(img_input, x, name='vgg19custom')
    return model

## prepare data

In [0]:
#split train test examples
if is_splited:
  dir_path_train = dir_path_DataSet + 'train/'
  dir_path_test =  dir_path_DataSet + 'test/'
  listFiles_train, Y_train, size_train = read_dir(dir_path_train)
  listFiles_test, Y_test, size_test = read_dir(dir_path_test)

  if has_val:
    dir_path_val =  dir_path_DataSet + '/val/'
    listFiles_val, Y_val, size_val = read_dir(dir_path_val)


else:
  listFiles, Y, size = read_dir(dir_path_DataSet)
  listFiles_train, Y_train, listFiles_test, Y_test = dir_train_test_split(listFiles, Y)
  size_train = len(listFiles_train)
  size_test = len(listFiles_test)
  has_val = False
    

In [0]:
len(listFiles_train)

5221

## generate vectors and save in files

In [0]:
def vector_generator(Y_, size, nbSaveFiles, listFiles, path_save, path_save_Y):
  for i in range(nbSaveFiles):
    X = create_zero_array(layer, n, size-i*nbImagesPerFile, nbImagesPerFile)
    
    X, Y = complete_array_cancer(listFiles, X, Y_, n, i*nbImagesPerFile)
    if not flag_VLAD:
      X = pool(X, pooling)
    np.save(path_save[i], X)
    np.save(path_save_Y[i], Y)
    del X
    print('{}th file saved'.format(i))
    
def get_model(layer):
  if flag_load_model:
    if model_with_structure:
      network = load_full_model(model_file_path)
    else:
      network = get_cnn_model(model_name)
      network.load_weights(model_file_path)
  else:
    network = VGG19(weights='imagenet')
  model = Model(inputs=network.input, outputs=network.get_layer(layer).get_output_at(-1))
  if n >= 1 : 
    model = to_fully_conv(model)
  return model

In [0]:
for layer in layers:
  for n in scales: 
    if vgg_multi_files:
      nbImagesPerFile, nbSaveFiles_train, nbSaveFiles_test, nbSaveFiles_val = cal_nbSaveFiles(layer, n, size_train, size_test)
    else:
      nbImagesPerFile, nbSaveFiles_train, nbSaveFiles_test, nbSaveFiles_val = max(size_train, size_test), 1, 1, 1
      if not has_val:
        nbSaveFiles_val = 0
      
    HEIGHT, WIDTH, CHANNELS = 224, 224, 3
    NUM_LABELS = len(set(Y_train))
    
    
    model = get_model(layer)
    print("layer = " + layer + " n = " + str(n) )
    
    path_save_train, path_save_Y_train, path_save_test, \
    path_save_Y_test, path_save_val, path_save_Y_val = get_vector_files_path(flag_VLAD, vgg_multi_files, 
                                                                             vgg_path, doc, layer, n, pooling, 
                                                                             nbSaveFiles_train, nbSaveFiles_test,
                                                                             nbSaveFiles_val)

    vector_generator(Y_train, size_train, nbSaveFiles_train, listFiles_train, path_save_train, path_save_Y_train)
    vector_generator(Y_test, size_test, nbSaveFiles_test, listFiles_test, path_save_test, path_save_Y_test)
    if has_val:
      vector_generator(Y_val, size_val, nbSaveFiles_val, listFiles_val, path_save_val, path_save_Y_val)
    

    del model
print('Done')

OSError: ignored

In [0]:
len(Y_train)

6400

# SVM without VLAD

In [0]:
def load_vectors(layer, n, size_train, size_test):
  nbImagesPerFile, nbSaveFiles_train, nbSaveFiles_test, nbSaveFiles_val = cal_nbSaveFiles(layer, n, size_train, size_test)
#   nbSaveFiles_train, nbSaveFiles_test, nbSaveFiles_val = 1, 1, 0
  path_save_train, path_save_Y_train, path_save_test, \
  path_save_Y_test, path_save_val, path_save_Y_val = get_vector_files_path(flag_VLAD, vgg_multi_files, 
                                                                             vgg_path, doc, layer, n, pooling, 
                                                                             nbSaveFiles_train, nbSaveFiles_test,
                                                                             nbSaveFiles_val)

  Y_train = np.load(path_save_Y_train[0])
  Y_test = np.load(path_save_Y_test[0])
  X_train = np.load(path_save_train[0])
  X_test = np.load(path_save_test[0])
  for i in range(1, nbSaveFiles_train):
    X_train = np.concatenate((X_train, np.load(path_save_train[i])))
    Y_train = np.concatenate((Y_train, np.load(path_save_Y_train[i])))
  for i in range(1, nbSaveFiles_test):
    X_test = np.concatenate((X_test, np.load(path_save_test[i])))
    Y_test = np.concatenate((Y_test, np.load(path_save_Y_test[i])))
  return X_train, X_test, Y_train, Y_test

In [0]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

if flag_VLAD == False:
  for layer in layers:
    for n in scales:
      print('layer: {}, scale: {}'.format(layer, n))
      X_train, X_test, Y_train, Y_test = load_vectors(layer, n, size_train, size_test)
      
#       normalization
      X_train = normalize(X_train, norm = 'l2', axis = 1)
      X_test = normalize(X_test, norm = 'l2', axis = 1)

      """SUPPORT VECTOR MACHINE"""

      parameters = {'C':[0.00001,0.0001,0.001,0.01, 0.1, 1, 10,100]}
      svc = svm.SVC(kernel='linear')
      clf = GridSearchCV(svc, parameters, cv=3)
      clf.fit(X_train, Y_train)
      
#       print('train set Accuracy: %.2f' % clf.score(X_train, Y_train))
      Y_pred = clf.predict(X_test)
      try:
        cm  = confusion_matrix(Y_test, Y_pred)
        tn, fp, fn, tp= cm.ravel()
        precision = tp / (tp + fp)
        recall =    tp / (tp + fn)
        accuracy = (tp + tn) / (tn + fp + fn + tp)

        print("Recall of the model is {:.5f}".format(recall))
        print("Precision of the model is {:.5f}".format(precision))
        print("Accuracy of the model is {:.5f}".format(accuracy))
      except:
        print(accuracy_score(Y_test, Y_pred))
        
      print('best params: {}'.format(clf.best_params_))
      print('Done')
      
print('Done')

      

layer: fc2, scale: 1
Recall of the model is 0.99487
Precision of the model is 0.70674
Accuracy of the model is 0.73878
best params: {'C': 100}
Done
Done


In [0]:
len(X_train)

5049

# SVM with VLAD
## Kmeans

In [0]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import sklearn
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pickle
import numpy as np

def save_file(path,obj):
  pickle_out = open(path,"wb")
  pickle.dump(obj, pickle_out)
  pickle_out.close()

In [0]:
for layer in layers:
  for n in scales:
    print("layer = " + layer + " n = " + str(n) )
    X=np.load(vgg_path + 'X_train_'+layer + '(' + str(n) + ').npy')

    #this just for miniMIT bc we don't have many descriptorts 
    if doc=='miniMIT_Etus':
      if len(X.shape) == 4:
        k_descriptors = X.shape[1]*X.shape[2]
      else:
        k_descriptors = 1

    #reshape (d1,d2,d3,d4) -> (d1,d2*d3,d4)
    if len(X.shape) == 4:
      d=X.shape[1]*X.shape[2]
      X = np.reshape(X,(X.shape[0],d,X.shape[3]))
    else:
      d = 1
      X = np.reshape(X,(X.shape[0],d,X.shape[1]))

    #we take only some discriptors
    if doc!='miniMIT_Etus':
      #take random descriptors
      indexes = np.arange(d)
      np.random.shuffle(indexes)
      indexes=indexes[:k_descriptors]
      X=X[:,indexes,:]

    if(n==0):
      x_all=X[:]
    else:
      x_all=np.concatenate((x_all,X),axis=1)
    del X

  x_all=np.reshape(x_all,(x_all.shape[0]*x_all.shape[1],x_all.shape[2]))
  
  x_all = sklearn.preprocessing.normalize(x_all, norm = 'l2', axis = 1)

  x_all = StandardScaler(with_std=False).fit_transform(x_all)

  x_all = sklearn.preprocessing.normalize(x_all, norm = 'l2', axis = 1)
  
  print("PCA fit transform ...")
  pca = PCA(n_components=128)
  x_train = pca.fit_transform(x_all)
  
  x_train = sklearn.preprocessing.normalize(x_train, norm = 'l2', axis = 1)

  #kmeans
  print("Kmean1 64 ...")
  kmeans1 = KMeans(n_clusters=64, random_state=0).fit(x_train)
  print("Kmean2 256 ...")
  kmeans2 = KMeans(n_clusters=256, random_state=0).fit(x_train)

  #save pca, kmeans  pickle
  save_file(kmeans_path+'_'+layer+'_'+'PCA.pickle',pca)
  save_file(kmeans_path+'_'+layer+'_'+'kmean1.pickle',kmeans1)
  save_file(kmeans_path+'_'+layer+'_'+'kmean2.pickle',kmeans2)
        
  del x_all
print("END")
      

layer = fc1 n = 0


FileNotFoundError: ignored

## VLAD

In [0]:
import sklearn
from sklearn.cluster import KMeans
import numpy as np
from sklearn.svm import SVC
import pickle
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

In [0]:
def VLAD(X,visualDictionary):

  predictedLabels = visualDictionary.predict(X)
  centers = visualDictionary.cluster_centers_
  labels=visualDictionary.labels_
  k=visualDictionary.n_clusters

  m,d = X.shape
  V=np.zeros([k,d])
  #computing the differences

  # for all the clusters (visual words)
  for i in range(k):
    # if there is at least one descriptor in that cluster
    if np.sum(predictedLabels==i)>0:
      # add the diferences
      V[i]=np.sum(X[predictedLabels==i,:]-centers[i],axis=0)

  V = V.flatten()
  # power normalization
  V = np.sign(V)*np.sqrt(np.abs(V))

  # L2 normalization
  V = V/np.sqrt(np.dot(V,V))
  return V
  
def get_vlad(layer,scales,kmean1,kmean2,pca,type_set="train"):
  for n in scales:
    print(n , layer)
    #laod_data
    if type_set=="train":
      x=np.load(vgg_path +'X_train_'+layer + '(' + str(n) + ').npy')
    elif type_set=="test":
      x=np.load(vgg_path +'X_test_'+layer + '(' + str(n) + ').npy')


    #reshape (d1,d2,d3,d4) -> (d1,d2*d3,d4)
    if len(x.shape) == 4:
      x = np.reshape(x,(x.shape[0],(x.shape[1]*x.shape[2]),x.shape[3]))
    else:
      x = np.reshape(x,(x.shape[0],1,x.shape[1]))

    #if x_all is empty : x_all take x else add data to x_all 
    if(n==0):
      x_all=x
    else:
      x_all=np.concatenate((x_all,x),axis=1)
                
    
    V_64=[]
    V_256=[]
    #for ech image
    for vec in x_all:
      #we take the first vec and we delete it from the data set: memory problem
      x_all=x_all[1:,...]
      #normalization l2
      vec = normalize(vec, norm = 'l2', axis = 1)
      #transformation with PCA
      vec=pca.transform(vec)
      #normalisation
      vec = normalize(vec, norm = 'l2', axis = 1)


      V_64.append(VLAD(vec,kmean1))
      V_256.append(VLAD(vec,kmean2))
    
    return V_64,V_256

In [0]:
for layer in layers:
  print('loading files')
  kmean1 = pickle.load(open(kmeans_path+'_'+layer+'_'+'kmean1.pickle', 'rb'))
  kmean2 = pickle.load(open(kmeans_path+'_'+layer+'_'+'kmean2.pickle', 'rb'))
  pca = pickle.load(open(kmeans_path+'_'+layer+'_'+'PCA.pickle', 'rb'))

  print('VLAD for train set')
  v_train_64,v_train_256=get_vlad(layer,scales,kmean1,kmean2,pca,type_set="train")
  v_train_64 = np.asarray(v_train_64)
  v_train_256= np.asarray(v_train_256)

  np.save(vlad_path+'_'+layer+'_'+'vlad_train(64).npy',v_train_64)
  np.save(vlad_path+'_'+layer+'_'+'vlad_train(256).npy',v_train_256)
  del v_train_64,v_train_256

  print('VLAD for test set')
  v_test_64,v_test_256=get_vlad(layer,scales,kmean1,kmean2,pca,type_set="test")
  v_test_64 = np.asarray(v_test_64)
  v_test_256= np.asarray(v_test_256)

  np.save(vlad_path+'_'+layer+'_'+'vlad_test(64).npy',v_test_64)
  np.save(vlad_path+'_'+layer+'_'+'vlad_test(256).npy',v_test_256)
  del v_test_64,v_test_256

  print("done")

0 block5_pool
1 block5_pool
2 block5_pool
(171, 8192)
0 block5_pool
1 block5_pool
2 block5_pool
(73, 8192)
done
0 fc1
1 fc1
2 fc1
(171, 8192)
0 fc1
1 fc1
2 fc1
(73, 8192)
done
0 fc2
1 fc2
2 fc2
(171, 8192)
0 fc2
1 fc2
2 fc2
(73, 8192)
done


## SVM

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np

In [0]:
for layer in layers:
  print('prediction SVM [%s]' %(layer))
  
  v_train_64 = np.load(vlad_path+'_'+layer+'_'+'vlad_train(64).npy')
  v_train_256= np.load(vlad_path+'_'+layer+'_'+'vlad_train(256).npy')
  v_test_64  = np.load(vlad_path+'_'+layer+'_'+'vlad_test(64).npy')
  v_test_256 = np.load(vlad_path+'_'+layer+'_'+'vlad_test(256).npy')

  y_train=np.load(vgg_path + 'Y_train_'+layer + '(' + str(0) + ').npy')
  y_test=np.load( vgg_path + 'Y_test_' +layer + '(' + str(0) + ').npy')
  
  parameters = {'C':[0.0001,0.001,0.01, 0.1, 1, 10,100]}
  
  print('SUPPORT VECTOR MACHINE 256')
  clf = SVC(kernel='linear')
  clf = GridSearchCV(clf, parameters, cv=3)
  clf.fit(v_train_256, y_train)
  print('Accuracy vlad 256: %.2f' % clf.score(v_test_256, y_test))
  
  print('SUPPORT VECTOR MACHINE 64')
  clf = SVC(kernel='linear')
  clf = GridSearchCV(clf, parameters, cv=3)
  clf.fit(v_train_64, y_train)
  print('Accuracy vlad 64: %.2f' % clf.score(v_test_64, y_test))        

(171, 8192)
(171, 32768)
SUPPORT VECTOR MACHINE 256
Accuracy vlad 256: 0.74
prediction SVM [block5_pool]
SUPPORT VECTOR MACHINE 64
Accuracy vlad 64: 0.77
(171, 8192)
(171, 32768)
SUPPORT VECTOR MACHINE 256
Accuracy vlad 256: 0.71
prediction SVM [fc1]
SUPPORT VECTOR MACHINE 64
Accuracy vlad 64: 0.77
(171, 8192)
(171, 32768)
SUPPORT VECTOR MACHINE 256
Accuracy vlad 256: 0.74
prediction SVM [fc2]
SUPPORT VECTOR MACHINE 64
Accuracy vlad 64: 0.79
